<a href="https://colab.research.google.com/github/obikatonie/Anti-Bully/blob/master/Project_CyberBully_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import files
uploaded = files.upload()

Saving combined_data.csv to combined_data.csv


In [0]:
import io
import pandas as pd
import numpy as np
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [0]:
combine_data = pd.read_csv(io.BytesIO(uploaded['combined_data.csv']))

In [0]:
combine_data.head()

,",TWEET,LABELS"
0,"168698,wpdash says use en dash hyphen script r..."
1,"87811,sbsylvester bitch pull hoes u dream,1"
2,"78619,goddamn crazy 666886111,1"
3,"40546,hi school expect,0"
4,"184753,attention whore,1"


In [0]:
X = combine_data['TWEET'].values
y = combine_data["LABELS"].values

In [0]:
#split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)

In [12]:
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [13]:
#testing if GPU is available
gpu = tf.test.is_gpu_available()
gpu

True

In [14]:
X_train

array(['support support accurate warner still releasing hd dvd quitting cold turkey however comment date exclusive needs stay well mention date movies hd dvd released later blu ray movies',
       'hi vellella see removed link oceana group finshing comapny see wrong link creating perfectly legitimate page company want link anything better',
       'suggestion images hi interested images agneta frieberg ways solve copyright problem best chance would fashion advertisements without copyright notifications commons tag commonstemplatepd us notice resolution would need good enough read fine print',
       ...,
       '1 gave references authors talk page 2 every phrase text referenced 3 forced revert reverting text asked question reverted recived answer quotes given according question simply dont read answer talk page',
       'july 2011 welcome wikipedia although everyone welcome contribute wikipedia least one recent edits one made joan cusack edit reverted appears unconstructive constructiv

In [0]:
max_len = 100 #maximum length of each sequence in our vocabulary
num_words = None #total number of words in our vocabulary
embedding_size = 100 #our embedding dimension to which each word will be represented

#create the Tokenizer class
tokens = Tokenizer(num_words = num_words)
tokens.fit_on_texts(X_train.astype(str)) #fit the tokenizer class to X_train

X_train_seq = tokens.texts_to_sequences(X_train.astype(str)) #convert the words into sequences of integers
X_test_seq = tokens.texts_to_sequences(X_test.astype(str)) #convert the words into sequences of integers

In [0]:
#this gives us the total number of unique words in our vocabulary
num_words = len(tokens.word_counts)

In [17]:
num_words

185433

In [18]:
#the size of our vocabulary
vocab_size = len(tokens.word_index) + 1
vocab_size

185434

In [0]:
#pad the sequences
#here we will pad our training and testing input sequences with zeros
#a sentence having a length less than max_len will be padded with zeros
#a sentence having a length more than max_len will be cut-off to max_len = 100
X_train_pad = pad_sequences(X_train_seq, maxlen = max_len, padding = "post")
X_test_pad = pad_sequences(X_test_seq, maxlen = max_len, padding = "post")

In [0]:
from keras.layers import Embedding, LeakyReLU
from keras.layers import LSTM, CuDNNGRU, CuDNNLSTM
from keras.layers import Dense, Bidirectional
from keras.layers import Activation, BatchNormalization
from keras.layers import Dropout
from keras.models import Sequential
from keras import optimizers
from keras import backend as K
from keras.engine.topology import Layer
#from keras import initializations
from keras import initializers, regularizers, constraints
from keras.layers.merge import concatenate
from keras.layers import Input
from keras.models import Model
from keras.models import model_from_json, load_model

In [22]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length = max_len))

if gpu: #if gpu is available it will use "CuDNNLSTM" layer
  model.add(CuDNNLSTM(512, return_sequences = True, kernel_initializer = "lecun_normal"))
  model.add(Bidirectional(CuDNNLSTM(512)))
else: #if gpu is not available it will use "LSTM" layer
  model.add(LSTM(512, return_sequences = True, kernel_initializer = "lecun_normal"))
  model.add(Bidirectional(LSTM(512)))
model.add(Dense(512))
model.add(LeakyReLU(alpha=0.2))
#model.add(BatchNormalization())
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

W0815 19:23:18.666823 140291355195264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0815 19:23:18.676721 140291355195264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0815 19:23:18.684377 140291355195264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0815 19:23:20.498067 140291355195264 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0815 19:23:23.125688 140291355195264 de

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          18543400  
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100, 512)          1257472   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1024)              4202496   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 24,528,681
Trainable params: 24,528,681
Non-trainable params: 0
________________________________________________________________

In [23]:
#fit the model onto the dataset
h = model.fit(X_train_pad, y_train, 
                         epochs = 10, 
                         batch_size = 32,
                         validation_data = (X_test_pad, y_test),
                         verbose = 1)

Train on 136277 samples, validate on 58405 samples
Epoch 1/10
136277/136277 [==============================] - 325s 2ms/step - loss: 0.1955 - acc: 0.9294 - val_loss: 0.1452 - val_acc: 0.9436
Epoch 2/10
136277/136277 [==============================] - 323s 2ms/step - loss: 0.1172 - acc: 0.9544 - val_loss: 0.1513 - val_acc: 0.9441
Epoch 3/10
136277/136277 [==============================] - 321s 2ms/step - loss: 0.0785 - acc: 0.9706 - val_loss: 0.1797 - val_acc: 0.9395
Epoch 4/10
136277/136277 [==============================] - 323s 2ms/step - loss: 0.0516 - acc: 0.9811 - val_loss: 0.1978 - val_acc: 0.9354
Epoch 5/10
136277/136277 [==============================] - 322s 2ms/step - loss: 0.0334 - acc: 0.9883 - val_loss: 0.2610 - val_acc: 0.9289
Epoch 6/10
136277/136277 [==============================] - 323s 2ms/step - loss: 0.0221 - acc: 0.9918 - val_loss: 0.3411 - val_acc: 0.9309
Epoch 7/10
136277/136277 [==============================] - 321s 2ms/step - loss: 0.0159 - acc: 0.9943 - val_

In [0]:
#the following function takes input a new sentence, tokens and the loaded model
#it outputs whether a new sentence is a Hate speech or not
def make_predictions(input_text, tokens, model):
    sent = np.array(input_text)
    sent = sent.reshape(1, )
    sent_to_seq = tokens.texts_to_sequences(sent.astype(str))
    sent_pad = pad_sequences(sent_to_seq, maxlen = max_len, padding = "post")
    predict = model.predict_classes(sent_pad)
    if predict == 1:
        print("1")
    else:
        print("0")
    return predict

In [27]:
#sample example text1
sample_sent1 = "you are such a hoe"
predictions = make_predictions(sample_sent1, tokens, model)

1


In [28]:
#sample example text2
sample_sent2 = "anyone else anyone else access page dickwad sajuro gay dumbass name unreasonable think tv guide power anything president bet know said mad season 2 spelled snott still would believe tv guide"
predictions = make_predictions(sample_sent2, tokens, model)

1


In [29]:
#sample example text4
sample_sent4 = "fuck the police. I hope they kill all them bitches"
predictions = make_predictions(sample_sent4, tokens, model)

1


In [30]:
#sample example text5
sample_sent5 = "attention whore"
predictions = make_predictions(sample_sent5, tokens, model)

1


In [31]:
#sample example text6
sample_sent5 = "i love you to the moon and back"
predictions = make_predictions(sample_sent5, tokens, model)

0
